# **Projet de Controle Continu : Analyse Météorol**

# **1. Chargement des données**

In [1]:
import pandas as pd
from pydantic import BaseModel, ValidationError
from typing import Optional

class MeteoData(BaseModel):
    # On suppose ici quelques colonnes typiques pour un dataset météorologique
    temperature: Optional[float]
    humidity: Optional[float]
    wind_speed: Optional[float]
    precipitation: Optional[float]

def charger_fichier(nom_fichier: str) -> Optional[pd.DataFrame]:

    try:
        # Chargement du fichier CSV
        df = pd.read_csv(nom_fichier)

        # Pour afficher les premières lignes du DataFrame
        print("Les 5 premières lignes du fichier :")
        print(df.head())

        # Pour afficher un résumé statistique des colonnes numériques
        print("\nRésumé statistique des colonnes numériques :")
        print(df.describe())

        return df

    except FileNotFoundError:
        print(f"Erreur : le fichier '{nom_fichier}' n'existe pas.")
    except pd.errors.ParserError:
        print(f"Erreur : le fichier '{nom_fichier}' ne peut pas être lu. Vérifiez le format du fichier.")
    except Exception as e:
        print(f"Une erreur inattendue s'est produite : {e}")

    return None


In [2]:
# Appel de la fonction
df = charger_fichier("meteo.csv")


Les 5 premières lignes du fichier :
         date  max_temperature  min_temperature  precipitation  wind_speed
0  2023-09-01             30.5             18.2            0.0        12.3
1  2023-09-02             31.0             19.1            0.5        14.2
2  2023-09-03             29.8             17.6            1.2        10.4
3  2023-09-04             28.3             16.5            0.0         8.1
4  2023-09-05             32.1             20.4            0.0        15.6

Résumé statistique des colonnes numériques :
       max_temperature  min_temperature  precipitation  wind_speed
count        30.000000        30.000000      30.000000   30.000000
mean         30.230000        18.943333       0.683333   12.450000
std           3.176981         3.055541       1.234304    2.932076
min          24.800000        14.700000       0.000000    6.900000
25%          27.700000        16.500000       0.000000   10.450000
50%          29.900000        18.300000       0.000000   12.350000

# **2. la classe `Meteo`**

In [3]:
from datetime import datetime

class Meteo:
    def __init__(self, date: str, temperature: float, pression: float, humidite: float):
        self.date = date
        self.temperature = temperature
        self.pression = pression
        self.humidite = humidite

    @property
    def date(self):
        return self._date

    @date.setter
    def date(self, value):
        # Validation de la date
        try:
            self._date = datetime.strptime(value, "%Y-%m-%d")
        except ValueError:
            raise ValueError("Date doit être au format 'YYYY-MM-DD'.")

    @property
    def temperature(self):
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if not -50 <= value <= 60:
            raise ValueError("La température doit être comprise entre -50 et 60 degrés Celsius.")
        self._temperature = value

    @property
    def pression(self):
        return self._pression

    @pression.setter
    def pression(self, value):
        if not 800 <= value <= 1100:
            raise ValueError("La pression doit être comprise entre 800 et 1100 hPa.")
        self._pression = value

    @property
    def humidite(self):
        return self._humidite

    @humidite.setter
    def humidite(self, value):
        if not 0 <= value <= 100:
            raise ValueError("L'humidité doit être comprise entre 0% et 100%.")
        self._humidite = value

    @classmethod
    # Ce code nous permet de crée un objet Meteo à partir d'une ligne de DataFrame.
    def from_csv_row(cls, row):
        return cls(date=row['date'],
                   temperature=row['temperature'],
                   pression=row['pression'],
                   humidite=row['humidite'])

# ce code nous permet de surcharge de l'opérateur < pour comparer deux objets Meteo par température.
    def __lt__(self, other):

        return self.temperature < other.temperature

    def __gt__(self, other):
        """Surcharge de l'opérateur > pour comparer deux objets Meteo par température."""
        return self.temperature > other.temperature

    def __repr__(self):
        return (f"Meteo(date={self.date.strftime('%Y-%m-%d')}, "
                f"temperature={self.temperature}, "
                f"pression={self.pression}, "
                f"humidite={self.humidite})")

# Exemple de création d'objets Meteo
meteo_1 = Meteo(date="2024-10-30", temperature=20.5, pression=1013, humidite=60)
meteo_2 = Meteo(date="2024-10-31", temperature=25.0, pression=1010, humidite=55)

# Comparaison des objets
if meteo_1 > meteo_2:
    print("La première journée est plus chaude que la deuxième.")
else:
    print("La deuxième journée est plus chaude que la première.")


La deuxième journée est plus chaude que la première.


# **3. Validation des données**

In [6]:
from pydantic import BaseModel, Field, validator
from datetime import datetime

class MeteoValidator(BaseModel):
    date: datetime
    temperature: float = Field(..., ge=-50, le=60)
    pression: float = Field(..., ge=800, le=1100)
    humidite: float = Field(..., ge=0, le=100)

# On va valider la date pour s'assurer qu'elle est au bon format
    @validator('date', pre=True)
    def parse_date(cls, value):
# Il faut accepter des dates sous forme de chaîne de caractères 'YYYY-MM-DD'
        if isinstance(value, str):
            try:
                return datetime.strptime(value, "%Y-%m-%d")
            except ValueError:
                raise ValueError("La date doit être au format 'YYYY-MM-DD'")
        elif isinstance(value, datetime):
            return value
        raise TypeError("La date doit être une chaîne de caractères ou un objet datetime")
# Ce code nous permet de valider que la température est dans les bornes [-50, 60]"""
    @validator('temperature')
    def validate_temperature(cls, value):
        if not (-50 <= value <= 60):
            raise ValueError("La température doit être comprise entre -50 et 60 degrés Celsius.")
        return value

# Ici on valide que la température est dans les bornes [-50, 60]
    @validator('pression')
    def validate_pression(cls, value):
        if not (800 <= value <= 1100):
            raise ValueError("La pression doit être comprise entre 800 et 1100 hPa.")
        return value

# Ici on valide que l'humidité est dans les bornes [0, 100]
    @validator('humidite')
    def validate_humidite(cls, value):
        if not (0 <= value <= 100):
            raise ValueError("L'humidité doit être comprise entre 0% et 100%.")
        return value

# Voici un exemple d'utilisation pour verifier
try:
    meteo_validator = MeteoValidator(date="01-2023-01", temperature=70, pression=1015, humidite=50)
    print(meteo_validator)
except ValueError as e:
    print("Erreur de validation:", e)


Erreur de validation: 2 validation errors for MeteoValidator
date
  Value error, La date doit être au format 'YYYY-MM-DD' [type=value_error, input_value='01-2023-01', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error
temperature
  Input should be less than or equal to 60 [type=less_than_equal, input_value=70, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/less_than_equal


<ipython-input-6-2996385674bc>:11: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('date', pre=True)
<ipython-input-6-2996385674bc>:23: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('temperature')
<ipython-input-6-2996385674bc>:30: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide fo

# **4. Tests unitaires avec `unittest**`

In [9]:
import unittest
from datetime import datetime
import pandas as pd



class TestMeteo(unittest.TestCase):
# Je vais faire un Test que la température hors bornes lève une erreur.
    def test_temperature_validation(self):
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=70, pression=1015, humidite=50)
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=-60, pression=1015, humidite=50)
# Test que la pression hors bornes lève une erreur.
    def test_pression_validation(self):
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=25, pression=750, humidite=50)
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=25, pression=1150, humidite=50)
# Test que l'humidité hors bornes lève une erreur.
    def test_humidite_validation(self):
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=25, pression=1015, humidite=120)
        with self.assertRaises(ValueError):
            Meteo(date="2023-01-01", temperature=25, pression=1015, humidite=-10)

# Test de la méthode from_csv_row pour créer un objet Meteo.
    def test_creation_from_csv_row(self):
        # Crée une ligne de DataFrame fictive
        data = {'date': "2023-01-01", 'temperature': 20.0, 'pression': 1010, 'humidite': 45}
        row = pd.Series(data)

        # Création de l'objet Meteo
        meteo_obj = Meteo.from_csv_row(row)

        # Vérification des valeurs
        self.assertEqual(meteo_obj.date, datetime.strptime("2023-01-01", "%Y-%m-%d"))
        self.assertEqual(meteo_obj.temperature, 20.0)
        self.assertEqual(meteo_obj.pression, 1010)
        self.assertEqual(meteo_obj.humidite, 45)

# Test des opérateurs de comparaison pour les objets Meteo.
    def test_comparison_operators(self):
        meteo_1 = Meteo(date="2023-01-01", temperature=20.0, pression=400, humidite=45)
        meteo_2 = Meteo(date="2023-01-02", temperature=25.0, pression=1015, humidite=50)

        # Test de l'opérateur >
        self.assertTrue(meteo_2 > meteo_1)
        # Test de l'opérateur <
        self.assertTrue(meteo_1 < meteo_2)
        # Test de l'égalité des températures
        meteo_3 = Meteo(date="2023-01-03", temperature=20.0, pression=1020, humidite=55)
        self.assertFalse(meteo_1 > meteo_3)
        self.assertFalse(meteo_1 < meteo_3)

if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)



E....
ERROR: test_comparison_operators (__main__.TestMeteo)
Test des opérateurs de comparaison pour les objets Meteo.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-9-98cfb28c2430>", line 47, in test_comparison_operators
    meteo_1 = Meteo(date="2023-01-01", temperature=20.0, pression=400, humidite=45)
  File "<ipython-input-3-9d2e67e49217>", line 7, in __init__
    self.pression = pression
  File "<ipython-input-3-9d2e67e49217>", line 39, in pression
    raise ValueError("La pression doit être comprise entre 800 et 1100 hPa.")
ValueError: La pression doit être comprise entre 800 et 1100 hPa.

----------------------------------------------------------------------
Ran 5 tests in 0.014s

FAILED (errors=1)
